# RAG 기초

## Enviornment (.env)

In [1]:
# .env 파일을 불러옵니다.
from dotenv import load_dotenv
load_dotenv()

True

## Tool 설정

In [2]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

In [3]:
from langchain_core.pydantic_v1 import BaseModel, Field

# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class Add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [Add, Multiply]

## Tool Calling / Function Calling


### 1) OpenAI

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm_with_tools = llm.bind_tools(tools=[add, multiply])

In [5]:
query = "What is 3 * 12? Also, what is 11 + 49?"

response = llm_with_tools.invoke(query)
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_FhCkTbiHarD5Gy4hoSCPiAHp', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_PsBJHWXkQZuarX7s2mqKMRRR', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 88, 'total_tokens': 137}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-387c9513-f8fb-41a2-9205-1fc2d23930c1-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_FhCkTbiHarD5Gy4hoSCPiAHp'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_PsBJHWXkQZuarX7s2mqKMRRR'}] usage_metadata={'input_tokens': 88, 'output_tokens': 49, 'total_tokens': 137}


In [6]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_FhCkTbiHarD5Gy4hoSCPiAHp'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_PsBJHWXkQZuarX7s2mqKMRRR'}]

In [8]:
llm_with_tools = llm.bind_tools(tools=[Add, Multiply])
response = llm_with_tools.invoke(query)
print(response)


content='' additional_kwargs={'tool_calls': [{'id': 'call_0XV4yzJNQzJTLbEmkYEGpKhY', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'Multiply'}, 'type': 'function'}, {'id': 'call_S3peK9N6vr2ZOMbiZV0wNDGn', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'Add'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 105, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-c282b7d7-0773-4a9d-8d7a-b09c5ebdd652-0' tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_0XV4yzJNQzJTLbEmkYEGpKhY'}, {'name': 'Add', 'args': {'a': 11, 'b': 49}, 'id': 'call_S3peK9N6vr2ZOMbiZV0wNDGn'}] usage_metadata={'input_tokens': 105, 'output_tokens': 50, 'total_tokens': 155}


In [9]:
response.tool_calls

[{'name': 'Multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_0XV4yzJNQzJTLbEmkYEGpKhY'},
 {'name': 'Add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_S3peK9N6vr2ZOMbiZV0wNDGn'}]

In [11]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_DLIr9feoUMXu2AxOslUapcp3', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'Multiply'}, 'type': 'function'}, {'id': 'call_Gy8hzf9D7AnTZSrtu4HblMlo', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'Add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 105, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-66d20e52-3444-4a1d-93db-baa8aecac7ae-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_DLIr9feoUMXu2AxOslUapcp3'}, {'name': 'Add', 'args': {'a': 11, 'b': 49}, 'id': 'call_Gy8hzf9D7AnTZSrtu4HblMlo'}], usage_metadata={'input_tokens': 105, 'output_tokens': 50, 'total_tokens': 155}),
 ToolMessage(content='36', tool_call_id='call_DLIr9feoUMXu2AxOslUapcp3'),
 ToolMessage(co

In [12]:
llm_with_tools.invoke(messages)

AIMessage(content='The result of 3 * 12 is 36, and the result of 11 + 49 is 60.', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 171, 'total_tokens': 197}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f36169bf-5415-45b8-be75-28e6871145d2-0', usage_metadata={'input_tokens': 171, 'output_tokens': 26, 'total_tokens': 197})

### 2) Ollama

In [29]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

llm = OllamaFunctions(model="llama3", format="json")
llm_with_tools = llm.bind_tools(tools=[Add, Multiply])

# query = "What is 3 * 12? Also, what is 11 + 49?"
query = "what is 11 + 49? Also, What is 3 * 12?"
response = llm_with_tools.invoke(query)
print(response)

content='' id='run-db56dbad-e61c-4be7-938a-a608e03363b5-0' tool_calls=[{'name': 'Add', 'args': {'a': 11, 'b': 49}, 'id': 'call_7b241f31111e4449bbe7d90b33b9f393'}]


In [30]:
response.tool_calls

[{'name': 'Add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_7b241f31111e4449bbe7d90b33b9f393'}]

In [33]:
from langchain_core.messages import HumanMessage, FunctionMessage

# messages = [HumanMessage(query)]
messages = []
ai_msg = llm_with_tools.invoke(query)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(FunctionMessage(tool_output, name=tool_call["name"]))
    
messages

[AIMessage(content='', id='run-b6ec5e12-1d95-460c-92b9-1dbaa5806a1d-0', tool_calls=[{'name': 'Add', 'args': {'a': 11, 'b': 49}, 'id': 'call_2d635c04e9d34b7e9ae7ca63ddf387a2'}]),
 FunctionMessage(content='60', name='Add')]

In [34]:
from langchain_community.llms import Ollama
Ollama(model="llama3").invoke(messages)

"It looks like you're trying to ask me a math question!\n\nThe function `f(x) = 60` is a constant function, which means that it always returns the same value, regardless of the input `x`.\n\nIn other words, for any value of `x`, the output of this function will be exactly `60`.\n\nIs there anything else you'd like to know or discuss?"

In [23]:
messages

[HumanMessage(content='what is 11 + 49? Also, What is 3 * 12?'),
 AIMessage(content='', id='run-7a75de94-4eed-4a67-9e18-5d62295519fc-0', tool_calls=[{'name': 'Add', 'args': {'a': 11, 'b': 49}, 'id': 'call_cf2f9077ed624807affc3b9a96ee148d'}]),
 FunctionMessage(content='60', name='Add')]

In [35]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

chat_prompt = ChatPromptTemplate.from_template("""Breifely answer the question based on the following operation results:
                                               {context}
                                               
                                               Question:{question}
                                               Answer:""")
chat_model = ChatOllama(model="llama3")
llm_chain = chat_prompt | chat_model| StrOutputParser()
llm_chain.invoke({"context":messages, "question":query})

'According to the operation results:\n\n* 11 + 49 = 60 (based on the "Add" function call)\n* 3 * 12 = ? (no information provided about this multiplication)\n\nSo, we can confidently answer the first question:\n\nWhat is 11 + 49? => 60\n\nHowever, we cannot provide an answer for the second question without additional information!'

In [37]:
def process_query(query):
    messages = []
    ai_msg = llm_with_tools.invoke(query)
    messages.append(ai_msg)
    for tool_call in ai_msg.tool_calls:
        selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
        tool_output = selected_tool.invoke(tool_call["args"])
        messages.append(FunctionMessage(tool_output, name=tool_call["name"]))   

    chat_prompt = ChatPromptTemplate.from_template("""다음 계산에 근거하여 질문에 간단히 한국어로 답하세요:
                                                {context}
                                                
                                                Question:{question}
                                                Answer:""")
    chat_model = ChatOllama(model="llama3")
    llm_chain = chat_prompt | chat_model| StrOutputParser()
    response = llm_chain.invoke({"context":messages, "question":query})
    if response:
        return response
    else:
        return "No response found."

query = "23 더하기 17은 얼마입니까?"
process_query(query)

'23 더하기 17은 40입니다. 😊'

## Structured Output

#### 1) (pydantic) Schema

In [1]:
from typing import Optional
from langchain_core.pydantic_v1 import BaseModel, Field

class Car(BaseModel):
    """Information about a car."""
    make: Optional[str] = Field(default=None, description="The make of the car")
    model_name: Optional[str] = Field(default=None, description="The model name of the car")
    model_year: Optional[int] = Field(
        default=None, description="The year the car model was manufactured"
    )
    color: Optional[str] = Field(default=None, description="The color of the car")
    price: Optional[float] = Field(default=None, description="The price of the car")
    mileage: Optional[float] = Field(default=None, description="The mileage of the car")


#### 2) Prompt

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        ("human", "{text}"),
    ]
)

#### 3) OpenAI

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

runnable = prompt | llm.with_structured_output(schema=Car)

In [6]:
text = """
현재 환경부와 산업통상자원부가 심사 중인 BYD의 소형 해치백 차량인 ‘돌핀’과 중형 세단 차량인 ‘씰’의 중국 내 최저 판매 가격은 각각 1900만 원, 3900만 원이다. 특히 돌핀은 국내에서 가장 값싼 경형 전기차인 ‘기아 레이EV(세제 혜택 전 2775만 원)’와 비교해도 압도적으로 저렴하다.
씰은 BYD의 셀투보디(CTB) 기술이 세계 최초로 적용된 차량으로 가격 대비 높은 성능을 자랑한다. CTB란 차량 본체와 배터리·배터리관리시스템(BMS) 등을 하나로 통합해 강성과 효율성을 모두 높이는 기술을 뜻한다. 두 차량 모두 유럽의 신차 안정성 프로그램(euro NCAP)에서 최고 등급을 받기도 했다.
한국 시장 진입을 위해 BYD가 현지 판매가와 유사한 수준으로 가격을 책정할 가능성도 있다. 통상 국내 시장 진입 시 가격을 더 높여잡는 게 일반적이지만 중국산 제품에 대한 한국의 부정적 인식을 고려해 가격 경쟁력을 최우선적으로 확보할 수 있다는 것이다. 스위스 투자은행(IB) UBS에 따르면 BYD는 배터리, 차량용 반도체, 소프트웨어 등 전체 부품 75%에 대한 수직 계열화를 이루면서 경쟁사 대비 30% 수준의 가격 우위를 확보하고 있다. 아울러 리튬·인산·철(LFP) 배터리에 대한 환경부의 불리한 규정에도 일정 수준의 보조금 확보도 가능하다. 현재 돌핀과 씰의 판매 가격은 국내 전기차 보조금 전액 지원 기준인 5500만 원을 충족한다. 유럽 인증 기준을 만족시키는 최대 427㎞(돌핀), 570㎞(씰)에 이르는 1회 충전 주행거리도 유리한 요소다.
BYD의 대항마로는 최근 기아가 출시한 소형 스포츠유틸리티차량(SUV) ‘EV3’가 꼽힌다. EV3는 니켈·코발트·망간(NCM) 배터리를 탑재해 롱레인지 모델 기준 1회 충전에 501㎞ 주행거리를 확보했다. 가격은 보조금 적용 시 3000만 원 중반대로 전기차 대중화라는 목표를 이루기 위한 기아의 주력 모델이다. KG모빌리티의 코란도EV(3000만 원대)도 BYD의 경쟁 상대다.
"""
response = runnable.invoke({"text": text})
print(response)

make='BYD' model_name='돌핀' model_year=None color=None price=19000000.0 mileage=None


In [3]:
from typing import List
class Data(BaseModel):
    """Extracted data about cars."""
    cars: List[Car] = Field(
        default=None, description="Extracted information about cars"
    )

#### 4) 여러 개의 Entity 추출

In [7]:
from typing import List
class Data(BaseModel):
    """Extracted data about cars."""
    cars: List[Car] = Field(
        default=None, description="Extracted information about cars"
    )

runnable = prompt | llm.with_structured_output(schema=Data)
response = runnable.invoke({"text": text})
print(response)

cars=[Car(make='BYD', model_name='돌핀', model_year=None, color=None, price=19000000.0, mileage=None), Car(make='BYD', model_name='씰', model_year=None, color=None, price=39000000.0, mileage=None), Car(make='기아', model_name='레이EV', model_year=None, color=None, price=27750000.0, mileage=None), Car(make='기아', model_name='EV3', model_year=None, color=None, price=30000000.0, mileage=None), Car(make='KG모빌리티', model_name='코란도EV', model_year=None, color=None, price=30000000.0, mileage=None)]


### 5) Ollama

In [15]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """<|start_header_id|>system<|end_header_id|>
You are an expert extraction algorithm. Only extract relevant information from the text.
If you do not know the value of an attribute asked to extract, return null for the attribute's value.
<|eot_id|><|start_header_id|>user<|end_header_id|>

TEXT: {context}
QUESTION: {question}
JSON:
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""
)

llm = OllamaFunctions(model="llama3", format="json", temperature=0)
chain = prompt | llm.with_structured_output(schema=Car)
response = chain .invoke({"context": text, "question": "Describe 코란도"})
print(response)

make=None model_name='코란도EV' model_year=None color=None price=3000.0 mileage=None


## API 호출

In [17]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Stock(BaseModel):
    """Trading Stock"""

    ticker: Optional[str] = Field(default=None, description="""The ticker of the stock ("005930", "AAPL", ...)""")
    start_date: Optional[str] = Field(default=None, description="""The start trading date ("2021-01-01", ...)""")
    end_date: Optional[str] = Field(default=None, description="""The end trading date ("2021-12-31", ...)""")

class Market(BaseModel):
    """Stock market index"""

    ticker: Optional[str] = Field(default=None, description="""The ticker of the market index based on the following list(KS11, VIX, ...):
        - KS11: KOSPI 지수, 코스피 지수
        - KQ11: KOSDAQ 지수, 코스닥 지수
        - KS200: KOSPI 200, 코스피 200
        - DJI: 다우존스 지수, Dow Jones Industrial Average
        - IXIC: 나스닥 종합지수, NASDAQ Composite
        - S&P500: S&P500 지수, NYSE
        - RUT: 러셀2000 지수, Russell 2000
        - VIX: VIX 지수, CBOE Volatility Index
        - SSEC: 상해 종합지수, Shanghai Composite
        - HSI: 항셍지수, Hang Seng
        - N225: 일본 닛케이지수, Nikkei 225
        - FTSE: 영국 FTSE100, FTSE 100
        - FCHI: 프랑스 CAC40 지수, CAC 40
        - GDAXI: 독일 닥스지수, DAX 30""")
    start_date: Optional[str] = Field(default=None, description="""The start trading date ("2021-01-01", ...)""")
    end_date: Optional[str] = Field(default=None, description="""The end trading date ("2021-12-31", ...)""")

In [18]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

llm = OllamaFunctions(model="llama3", format="json")
llm_with_tools = llm.bind_tools(tools=[Stock, Market])

query = "삼성전자의 2023년 주가"

response = llm_with_tools.invoke(query)
print(response.tool_calls)

[{'name': 'Stock', 'args': {'ticker': '005930', 'start_date': '2023-01-01', 'end_date': '2023-12-31'}, 'id': 'call_f86d6f49eac64fe583ad5a9d17343367'}]


In [19]:
query = "2024년 3월 테슬라"

response = llm_with_tools.invoke(query)
print(response.tool_calls)

[{'name': 'Stock', 'args': {'ticker': 'TSLA', 'start_date': '2024-03-01', 'end_date': '2024-03-31'}, 'id': 'call_3014e3c100594fde9b71e1d00b80b0f5'}]


In [21]:
query = "2022년 나스닥 지수"

response = llm_with_tools.invoke(query)
print(response.tool_calls)

[{'name': 'Market', 'args': {'ticker': 'IXIC', 'start_date': '2022-01-01', 'end_date': '2022-12-31'}, 'id': 'call_5e82cdcdaa2f413b9decf5233df0cf9e'}]


In [22]:
query = "코스닥 지수 2019년 7월"

response = llm_with_tools.invoke(query)
print(response.tool_calls)

[{'name': 'Market', 'args': {'ticker': 'KQ11', 'start_date': '2019-07-01', 'end_date': '2019-07-31'}, 'id': 'call_0d1ba771788d4ec6be5c938e7b1eae9e'}]


In [23]:
### FinanceDataReader
import FinanceDataReader as fdr

df = fdr.DataReader("KQ11", "2019-07-01", "2019-07-31")
df.head()

,Open,High,Low,Close,Volume,Change,UpDown,Comp,Amount,MarCap
Date,,,,,,,,,,
2019-07-01,696.38,697.97,692.76,696.00,734433278,0.0079,1,5.47,5156354340512,238458830094666
2019-07-02,695.80,699.44,693.19,696.25,696445726,0.0004,1,0.25,4052249681800,238565438926786
2019-07-03,697.68,698.89,689.86,693.04,676298786,-0.0046,2,-3.21,3979382946906,237389805604275
2019-07-04,693.96,694.63,682.73,691.27,763516530,-0.0026,2,-1.77,4132922883851,236950136647970
2019-07-05,692.86,694.39,686.93,694.17,729813499,0.0042,1,2.90,4048011198486,237965631550822


In [24]:
query = "2022년 3월 1일부터 2022년 3월 15일까지 삼성전자 주가"

ai_msg = llm_with_tools.invoke(query)
ticker = ai_msg.tool_calls[0]['args']['ticker']
start_date = ai_msg.tool_calls[0]['args']['start_date']
end_date = ai_msg.tool_calls[0]['args']['end_date']

print(ticker, start_date, end_date)
df = fdr.DataReader(ticker, start_date, end_date)
df.head


005930 2022-03-01 2022-03-15


,Open,High,Low,Close,Volume,Change
Date,,,,,,
2022-03-02,72300,72400,71500,71700,12481430,-0.005548
2022-03-03,72300,73100,72200,72900,13232638,0.016736
2022-03-04,72700,72700,71200,71500,13409634,-0.019204
2022-03-07,70000,70600,69900,70100,18617138,-0.019580
2022-03-08,68800,70000,68700,69500,15828269,-0.008559
